In [ ]:
from elasticsearch import Elasticsearch

: 

In [6]:
es_client = Elasticsearch("http://localhost:9200")

In [7]:
es_client.info()

ObjectApiResponse({'name': 'ff706f8e225b', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'IboGTbEZTLK7Dv3JPPr5pw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [8]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    print(course_name)

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

data-engineering-zoomcamp
machine-learning-zoomcamp
mlops-zoomcamp


In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.get(index=index_name, body=index_settings).body

{'course-questions': {'aliases': {},
  'mappings': {'properties': {'course': {'type': 'keyword'},
    'question': {'type': 'text'},
    'section': {'type': 'text'},
    'text': {'type': 'text'}}},
  'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'course-questions',
    'creation_date': '1719784368020',
    'number_of_replicas': '0',
    'uuid': 'cipTPOXKRL-4iaDTqWOQWw',
    'version': {'created': '8040399'}}}}}

In [11]:
from tqdm.auto import tqdm


/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:25<00:00, 37.02it/s]


In [13]:
query="How do I execute a command in a running docker container?"

In [14]:
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit["_score"])
result_docs

[84.220634, 84.220634, 84.220634]

In [15]:
def search_retriever(query, index_name="course-questions", max_results=3):
    search_query = {
        "size":  max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)

    return response
        

In [16]:
user_question = "How do I execute a command in a running docker container?" 
search_response = search_retriever(user_question)

context_template = """
Q: {question}
A: {text}
""".strip()

context_pieces = []
for hit in search_response['hits']['hits']:
    doc = hit['_source']
    context_piece = context_template.format(**doc)
    context_pieces.append(context_piece)

context = '\n\n'.join(context_pieces)
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
dock

In [17]:
def prompt_constructor(question, context):

    prompt_template = f"""
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()
    return prompt_template

In [18]:
user_question = "How do I execute a command in a running docker container?" 
prompt = prompt_constructor(user_question, context)
print(len(prompt))
print(prompt)

1347
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: How do I execute a command in a running docker container?
    
    CONTEXT: 
    Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How

In [19]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [20]:
tokens = encoding.encode(prompt)

In [21]:
len(tokens)

305

In [24]:
tokens[10:20]


[402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 271]

In [23]:
encoding.decode(tokens[10:20])

' on the CONTEXT from the FAQ database.\n   '

In [2]:
encoding.decode_single_token_bytes(63842)


b"You're"